# Samay Techinical Assesstment 

The following assestment aims to develop, and optimize a machine learning solution for detecting and classifying respiratory diseases. This will be achieved using a dataset of lung sounds recorded with an electronic stethoscope. The comprehensive IPython Notebook delivered should demonstrate the candidate's proficiency in analyzing sensor data for health applications, specifically in the context of pulmonary diseases.


**Primary Task :** Conduct a thorough analysis of the lung sound dataset, extracting meaningful patterns and insights relevant to respiratory diseases. This analysis should go beyond basic statistical descriptions and delve into the nuances and complexities of the data.

## Loading the data

The data set of the audio cames from tow different sources, please clone the respository:
https://github.com/czhangg17/Lung-Disease-detection

Create a branch with your changes

In [1]:
#!pip3 install tensorflow

In [ ]:
# libraries

import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
import pandas as pd

#from tensorflow.keras import layers, models

from IPython import display

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import matplotlib.image as mpimg
#import keras
#from keras.models import Sequential
#from tensorflow.keras.optimizers import Adam
#from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense
#from sklearn.utils import shuffle
#from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa
import cv2

import ntpath
import random
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
file_paths = []
for dirname, _, filenames in os.walk('./Audio Files/'):
    for filename in filenames:
        file_paths.append(os.path.join(dirname,filename))
        
        
import scipy
import scipy.io
import scipy.io.wavfile
from scipy import signal
from scipy.fft import fftshift
from os.path import dirname, join as pjoin
from scipy.io import wavfile

In [5]:
os.chdir('/Users/andresheredialopez/Documents/Cindy/Assesstment_ML/Lung-Disease-detection') # change your path

The followign functions will help you to read and spectogram and the audio file

### To do:
Feel free to use it , or create you own functions to read the data, make some graphics to understand what is inside the data

### glue:
 
the library display play the ayudio playback, you can use the command : display.display(display.Audio(data, rate=16000))

In [7]:
def get_spectrogram(waveform):
    
  # Zero-padding for an audio waveform with less than 16,000 samples.
    input_len = 16000
    waveform = waveform[:input_len]
    zero_padding = tf.zeros(
      [16000] - tf.shape(waveform),
      dtype=tf.float32)
  # Cast the waveform tensors' dtype to float32.
    waveform = tf.cast(waveform, dtype=tf.float32)
  # Concatenate the waveform with `zero_padding`, which ensures all audio
  # clips are of the same length.
    equal_length = tf.concat([waveform, zero_padding], 0)
  # Convert the waveform to a spectrogram via a STFT.
    spectrogram = tf.signal.stft(
    equal_length, frame_length=255, frame_step=128)
  # Obtain the magnitude of the STFT.
    spectrogram = tf.abs(spectrogram)
  # Add a `channels` dimension, so that the spectrogram can be used
  # as image-like input data with convolution layers (which expect
  # shape (`batch_size`, `height`, `width`, `channels`).
    spectrogram = spectrogram[..., tf.newaxis]
    return spectrogram

In [8]:
def plot_spectrogram(spectrogram, ax):
    if len(spectrogram.shape) > 2:
        
        assert len(spectrogram.shape) == 3
        spectrogram = np.squeeze(spectrogram, axis=-1)
  # Convert the frequencies to log scale and transpose, so that the time is
  # represented on the x-axis (columns).
  # Add an epsilon to avoid taking a log of zero.
    log_spec = np.log(spectrogram.T + np.finfo(float).eps)
    height = log_spec.shape[0]
    width = log_spec.shape[1]
    X = np.linspace(0, np.size(spectrogram), num=width, dtype=int)
    Y = range(height)
    ax.pcolormesh(X, Y, log_spec)

In [9]:
def read_wave(wav_fname):
    samplerate, data = wavfile.read(wav_fname)
    print("sample rate : ",samplerate)
    print("data : ",data)
    print("data shape : ",data.shape)
    print(f"number of channels = {data.shape[0]}")
    # number of channels = 2

    length = data.shape[0] / samplerate

    print(f"length = {length}s")
    # length = 0.01s

In [12]:
# and example of an wav data
samplerate, data = wavfile.read('./Audio Files/EP81_N,N,P L U,33,M.wav')
wav_fname='./Audio Files/EP81_N,N,P L U,33,M.wav'

In [ ]:
samplerate, data = wavfile.read(wav_fname)
print("sample rate : ",samplerate)
print("data : ",data)
print("data shape : ",data.shape)
print(f"number of channels = {data.shape[0]}")
    # number of channels = 2

length = data.shape[0] / samplerate

print(f"length = {length}s")
    # length = 0.01s

In [ ]:
# plotting and spectogram 
fig, axes = plt.subplots(2, figsize=(12, 8))
timescale = np.arange(data.shape[0])
axes[0].plot(timescale, data)
# axes[0].set_title('Waveform')
axes[0].set_xlim([0, 16000])

plot_spectrogram(spectrogram.numpy(), axes[1])
# axes[1].set_title('Spectrogram')
plt.show()


### Descriptive analysis and feature engenering

Many times, in audio processing the matrix of the date is treat as a normal dataset for each sample, descriptive statistics is usefull to understand the variability of energy. This should be the fisrts approach befere start modelling


## To do:

After you extract the audio matrix, create a function that returns mean,max, min, median, standar deviation and variation coefficient in a table.

You will see that names of the audio files contains a label of the sample, you will need to clean the file name and extract the lables. Some of the labels should be in the lables object. Use ',' and '-' separtion.

In [ ]:
label_expected={'Asthma',
 'Asthma and lung fibrosis',
 'BRON',
 'COPD',
 'Heart Failure',
 'Heart Failure + COPD',
 'Heart Failure + Lung Fibrosis ',
 'Lung Fibrosis',
 'N',
 'Plueral Effusion',
 'asthma',
 'copd',
 'heart failure',
 'pneumonia'}

## To do:
Explain a way to encode the labels and why will you use it

### Feature engenering
The feature engineering is the way to create acoustic features of the data, many times audio features are used to model instead of the raw data, so a preprocessing is need.

## To do:

Extract relevant features from the audio data, with a focus on entropy-based representations, such as Shannon entropy.
Analyze the impact of different filters (Bell, Diaphragm, and Extended modes) and recording locations on feature selection.


In [ ]:
# this is an example

spectrogram = get_spectrogram(data)
spectrogram

# Machine Learning Algorithm Design and Validation


Primary Task: Develop a machine learning algorithm using the provided dataset of lung sounds. This model should be capable of accurately detecting and classifying respiratory diseases.

Approach:
Data Preprocessing and Feature Engineering: 
Extract relevant features from the audio data, with a focus on entropy-based representations, such as Shannon entropy.
Analyze the impact of different filters (Bell, Diaphragm, and Extended modes) and recording locations on feature selection.
Algorithm Design: Design the algorithm considering various approaches to handling biomedical data. Ensure the algorithm efficiently processes the preprocessed and feature-engineered data to achieve accurate classification results.
Innovative Modeling Techniques: Utilize advanced machine learning techniques, exploring options like ensemble methods, decision trees, and neural networks. The candidate should demonstrate creativity and proficiency 